<h1>This is the code for COM3013 Coursework</h1>
<h2>Mofe Awosanya, Ebose Odijie, Hishaam Khan</h2>

These are all the imports

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Optimizer
import random

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

In [ ]:
import torch.multiprocessing as mp

# Set the sharing strategy
mp.set_sharing_strategy('file_system')

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

x_list = []
y_list = []

for inputs, labels in trainloader:
    x_list.append(inputs)
    y_list.append(labels)

x = torch.cat(x_list, dim=0)
y = torch.cat(y_list, dim=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()



dataiter = iter(trainloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(256 * 4 * 4, 120)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu5 = nn.ReLU()
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = x.view(-1, 256 * 4 * 4)
        x = self.relu4(self.fc1(x))
        x = self.relu5(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
model = Model()
model = model.float()

print(model)

In [ ]:
loss_func = torch.nn.CrossEntropyLoss()
loss_values = []

In [ ]:
default_optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        outputs = model(inputs)
        loss = loss_func(outputs, labels)
        default_optimizer.zero_grad()
        loss.backward()
        default_optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
            print('fc3 Weights: ')

print('Finished using initial SGD Optimization')

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4))) 

In [ ]:
correct = 0
total = 0

gradient_descent_data = []
i=0

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        gradient_descent_data.append((i, 100 * correct // total))
        i+=1


print(f'Model Accuracy on Testing Images: {100 * correct // total} %')

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
for param in model.fc3.parameters():
    param.requires_grad = True

In [ ]:
# import torch.nn.init as init

# def initialise_weights(layer):
#     if isinstance(layer, nn.Linear):
#         init.xavier_uniform_(layer.weight)
#         nn.init.zeros_(layer.bias)
        
# model.fc3.apply(initialise_weights)

In [ ]:
PATH = './cifar_model.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
from deap import creator, base, tools, algorithms
from sympy.combinatorics.graycode import GrayCode
from sympy.combinatorics.graycode import gray_to_bin
from sympy.combinatorics.graycode import bin_to_gray

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

pop_size = 5 #population size
num_of_bits = 8 #number of bits in a chromosome
iterations = 5 #number of generations
dspInterval = 1

cross_prob = 0.75 #crossover probability
mutate_prob = 0.95 #mutation probability
num_elitists = 5 #the number of elite individuals selected

maxnum = 2*num_of_bits

fc1shape = 256*4*4*120
fc2shape = 120*84
fc3shape = 84*10

chromosome_length = (84*10)+1

flip_prob = 0.1

toolbox = base.Toolbox()
# toolbox.register("attr_bool", random.randint, 0, 1)
# toolbox.register("individual", tools.initRepeat, creator.Individual, np.random.randint, 2, num_of_bits)
# toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# toolbox.register("evaluate", evaluate)
# toolbox.register("mate", tools.cxTwoPoint)
# toolbox.register("mutate", tools.mutFlipBit, indpb=flip_prob)
# toolbox.register("select", tools.selTournament, fit_attr="fitness")

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, num_of_bits*(chromosome_length))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def retrieve_weights():
    fc3list = model.fc3.weight.reshape(fc3shape).tolist()

    fc3bias = model.fc3.bias.tolist()

    weights = fc3list + fc3bias

    return weights

def real_to_chromosome(weight):
    weight = weight/20
    if weight < -1:
        weight = -1
    if weight > 1:
        weight = 1
    integerPart = int(maxnum * (weight + 1 ) / 2)
    if (integerPart == maxnum):
        integerPart -= 1
    chromosome = [int(d) for d in str(bin(integerPart))[2:]]
    while (len(chromosome) < num_of_bits):
        chromosome.insert(0, 0)
    indasstring=''.join(map(str, chromosome))
    chromosome=bin_to_gray(indasstring)
    output=[]
    for digit in chromosome:
        output.append(int(digit))

    return output

def chromosome_to_real(c):
    indasstring=''.join(map(str, c))
    degray=gray_to_bin(indasstring)
    numasint=int(degray, 2)
    numinrange=-1.0+2.0*numasint/maxnum
    return numinrange*20

def separatevariables(v):
    sep = []
    for i in range (0,num_of_bits*(chromosome_length),num_of_bits):
        sep.append(chromosome_to_real(v[i:i+num_of_bits]))
    return sep

In [ ]:
def calc_fitness(individual):
    weights = separatevariables(individual)
    weights = np.asarray(weights)

    model.fc3.weight = torch.nn.Parameter(torch.from_numpy(weights[:fc3shape].reshape(84, 10).T).float())
    model.fc3.bias = torch.nn.Parameter(torch.from_numpy(weights[[chromosome_length - 1]]).float())

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in trainloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return 1 - accuracy

    

toolbox.register("evaluate", calc_fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=flip_prob)
toolbox.register("select", tools.selTournament, fit_attr="fitness")

In [ ]:
popa = toolbox.population(n=pop_size)
pop=[]
for individual in popa:
    sep = separatevariables(individual)
    weightlist=[]
    for weight in sep:
        weightlist+=real_to_chromosome(weight)
    pop.append(creator.Individual(weightlist))

fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = (fit,)

print(" Evaluated %i individuals" % len(pop))

fits = [ind.fitness.values[0] for ind in pop]

g = 0

while g < iterations:
    g = g + 1
    print("-- Generation %i -- " % g)

    offspring = tools.selBest(pop, num_elitists)
    offspring += toolbox.select(pop, len(pop)-num_elitists, 2)
    offspring = list(map(toolbox.clone, offspring))
    randomnum = random.random()
    print(randomnum)

    for child1, child2 in zip(offspring[:-1:2], offspring[1::2]):
        if randomnum < cross_prob:
            print("Crosssover")
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
        else:
            print("No Crossover")
            
            
    for mutant in offspring[2::]:
        if randomnum < mutate_prob:
            print("Mutation")
            toolbox.mutate(mutant)
            del mutant.fitness.values
        else:
            print("No Mutation")

    for individual in offspring:
        weights=separatevariables(individual)
        weights=np.asarray(weights)
    
        model.fc3.weight = torch.nn.Parameter(torch.from_numpy(weights[:fc3shape].reshape(84, 10).T).float())
        model.fc3.bias = torch.nn.Parameter(torch.from_numpy(weights[[chromosome_length - 1]]).float())

        for epoch in range(2):
            for inputs, labels in trainloader:
                default_optimizer.zero_grad()
                outputs=model(inputs)
                loss=loss_func(outputs, labels)
                # Print the loss for each iteration
                loss.backward()
                default_optimizer.step()
            print(f'epoch %i' % epoch)

        outputs=model(inputs)
        loss=loss_func(outputs, labels)
        print('Post loss.item:', loss.item())
        fc3list = model.fc3.weight.reshape(fc3shape).tolist()
        fc3bias = model.fc3.bias.reshape(1).tolist()
        weights = fc3list + fc3bias

        individual.clear()
        newInd=[]
        for index in range(len(weights)):
            chromosome = real_to_chromosome(weights[index])
            newInd += chromosome
        newInd = creator.Individual(newInd)
        offspring.remove(individual)
        offspring.append(newInd)
    
    pop[:] = offspring
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = (fit,)

    fits = [ind.fitness.values[0] for ind in pop]
    loss_values.append(min(fits))

    if g%dspInterval == 0:
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print(" Min %s" % min(fits))
        print(" Max %s" % max(fits))
        print(" Avg %s" % mean)
        print(" Std %s" % std)

print("-- Completed Evolution --")

best_individual = tools.selBest(pop, 1)[0]
print("Loss of Genetic Algorithm Optimiser on Final Layer: %s" % (calc_fitness(best_individual)))

In [ ]:
weights=separatevariables(best_individual)
weights=np.asarray(weights)

model.fc3.weight = torch.nn.Parameter(torch.from_numpy(weights[:fc3shape].reshape(84, 10).T).float())
model.fc3.bias = torch.nn.Parameter(torch.from_numpy(weights[[chromosome_length - 1]]).float())

genetic_algorithm_data = []

i = 0
# Test the accuracy on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        genetic_algorithm_data.append((i, 100 * correct // total))
        i += 1

print(f'Model Accuracy on Testing Images: {100 * correct // total} %')

In [ ]:
population = toolbox.population(n=pop_size)
pop=[]
for individual in population:
    sep = separatevariables(individual)
    print(sep)
    weightlist=[]
    for weight in sep:
        weightlist+=real_to_chromosome(weight)
    pop.append(creator.Individual(weightlist))

fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

while g < iterations:
    g = g + 1
    print("-- Generation %i -- " % g)
    
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))
    
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cross_prob:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
            
    for mutant in offspring:
        if random.random() < mutate_prob:
            toolbox.mutate(mutant)
            del mutant.fitness.values
            
    # Replace the old population with the offspring
    population[:] = offspring
    
    if g%dspInterval == 0:
        fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print(" Min %s" % min(fits))
    print(" Max %s" % max(fits))
    print(" Avg %s" % mean)
    print(" Std %s" % std)
    
best_individual = tools.selBest(population, k=1)[0]


    

In [ ]:
def calc_loss(model, data_loader, loss_func):
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            running_loss += loss.item()
    return loss.item()

In [ ]:
def update_model_weights(model, weights):
    weights = np.asarray(weights)
    
    model.fc3.weight = torch.nn.Parameter(torch.from_numpy(weights[:fc3shape].reshape(84, 10).T).float())
    model.fc3.bias = torch.nn.Parameter(torch.from_numpy(weights[[chromosome_length - 1]]).float())

In [ ]:
def pattern_search(model, trainloader, loss_func, initial_delta=1.0, delta_decay=0.5, max_iter=10, min_delta=1e-6):
    best_loss = calc_loss(model, trainloader, loss_func)
    current_weights = retrieve_weights()
    delta = initial_delta

    for iteration in range(max_iter):
        improved = False

        # Explore adding delta
        new_weights = [w + delta for w in current_weights]
        update_model_weights(model, new_weights)
        loss = calc_loss(model, trainloader, loss_func)
        if loss < best_loss:
            best_loss = loss
            current_weights = new_weights
            improved = True

        # Explore subtracting delta
        new_weights = [w - delta for w in current_weights]
        update_model_weights(model, new_weights)
        loss = calc_loss(model, trainloader, loss_func)
        if loss < best_loss:
            best_loss = loss
            current_weights = new_weights
            improved = True

        if not improved:
            delta *= delta_decay  # Reduce delta if no improvement

        # Print progress
        progress_percentage = (iteration + 1) / max_iter
        print(f'Iteration {iteration}, Best Loss: {best_loss}, Delta: {delta}, Progress: {progress_percentage:.2f}%')

        # Stopping criterion based on delta
        if delta < min_delta:
            print(f'Stopping criterion met: Delta is below {min_delta}')
            break

    # Set model weights to the best found
    update_model_weights(model, current_weights)
    print("Loss of Pattern Search Optimiser on the Final Layer: ", best_loss)

In [ ]:
# Run pattern search
pattern_search(model, trainloader, loss_func)

In [ ]:
correct = 0
total = 0

pattern_search_data = []
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pattern_search_data.append((i, 100 * correct // total))
        i+=1

print(f'Model Accuracy on Testing Images: {100 * correct // total} %')

In [ ]:
evaluations_gradient, accuracy_gradient = zip(*gradient_descent_data)
evaluations_psearch, accuracy_psearch = zip(*pattern_search_data)
evaluations_genetic, accuracy_genetic = zip(*genetic_algorithm_data)

plt.figure(figsize=(10,6))

plt.plot(evaluations_gradient, accuracy_gradient, label="SGD")
plt.plot(evaluations_psearch, accuracy_psearch, label="Pattern Search")
plt.plot(evaluations_genetic, accuracy_genetic, label="Genetic Algorithm")


plt.title('Performance Trend of Learning Methods')
plt.xlabel('Number of Evaluations of the Objective Function')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()